In [1]:
!pip install -qqq -U pip
!pip install -qqq bitsandbytes --progress-bar off
!pip install -qqq torch --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@main --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git --progress-bar off
!pip install -qqq accelerate --progress-bar off
!pip install -qqq datasets --progress-bar off
!pip install -qqq loralib --progress-bar off
!pip install -qqq einops --progress-bar off
!pip install -qqq sentencepiece --progress-bar off

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import gc
import torch

# model.to("cpu")
# del model
torch.cuda.empty_cache()
gc.collect()

30

In [3]:
!nvidia-smi

Sun Feb 23 08:01:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   33C    P0             50W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
notebook_login()

# URDU MODEL

In [5]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [6]:
from unsloth import FastLanguageModel
import torch

dtype = None
load_in_4bit = True


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.1-8b-Instruct",
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "",
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.50.0.dev0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [7]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.2.15 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [8]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
EOS_TOKEN

'<|eot_id|>'

In [9]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "unsloth/llama-3.1-8b-instruct-unsloth-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128009,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": null,
    "llm_int8_threshold": 6.0,
    "load_in_4bit": true,
    "load_in_8bit": false,
    "quant_method": "bitsandbytes

In [10]:
import pandas as pd
df = pd.read_csv('/content/mongodb_prompts_with_stories(1-900).csv')
df.head()

,Story ID,Annotator ID 1,Annotator ID 3,Annotator ID 4,Story
0,1,ایسی کہانی لکھیں جس میں ایک عورت اپنی برابری ا...,یہ کہانی وجے رانی کی ہے جو برابری کی ڈھونڈ میں...,ایسی کہانی لکھو جس میں ایک عورت اپنی برابری او...,شام دبے پاؤں رینگ رہی تھی۔ ٹیلے پر درختوں کے س...
1,2,ایسی کہانی لکھیں جس میں ایک نوجوان لڑکی، جو قح...,یہ کہانی ایک بنگالی عورت سکینہ کی زندگی کے گرد...,ایسی کہانی لکھو جس میں ایک نوجوان لڑکی سکینہ ک...,جموں توی کے راستے کشمیر جائیے تو کُد کے آگے ای...
2,3,ایسی کہانی لکھیں جس میں ایک ریٹائرڈ سرکاری ملا...,یہ کہانی ایک ریٹائرڈ سرکاری ملازم حسین احمد کی...,ایسی کہانی لکھو جس میں ایک ریٹائرڈ ملازم کا کر...,رحمت خاں خاصا طویل تھا۔ دور دور تک پرانے مکانو...
3,4,ایسی کہانی لکھیں جس میں ایک تجربہ کار اور بہاد...,کشمیر کی لڑائی میں دو پرانے دوست، صوبیدار رب ن...,ایسی کہانی لکھو جس میں ایک فوجی اپنے پرانے دوس...,یہ کشمیر کی لڑائی بھی عجیب وغریب تھی۔ صوبیدار ...
4,5,ایسی کہانی لکھیں جس میں ایک طوائف کی زندگی کے ...,عیدن بائی کی کہانی ایک دلچسپ اور رومانوی کہانی...,ایسی کہانی لکھو جس میں ایک طوائف کی بیٹی اپنی ...,عیدن بائی آگرے والی، چھوٹی عید کو پیدا ہوئی تھ...


In [11]:
df.shape

(1000, 5)

In [12]:
# Define column names as variables
story_column = "Story"  # Update with actual column name
prompt_column_1 = "Annotator ID 1"  # Update with actual column name
prompt_column_2 = "Annotator ID 3"  # Update with actual column name
prompt_column_3 = "Annotator ID 4"  # Update with actual column name

In [13]:
# Create a new DataFrame where each prompt/story pair becomes an input-output pair
data = []

# Iterate through each row and create prompt-story pairs
for index, row in df.iterrows():
    # Each prompt and its corresponding story will be a training example
    for prompt_col in [prompt_column_1, prompt_column_2, prompt_column_3]:
        data.append({
            "input_text": row[prompt_col],
            "output_text": row[story_column]
        })

# Convert the list to a pandas DataFrame
train_df = pd.DataFrame(data)

In [14]:
train_df=train_df.dropna()

In [15]:
train_df.head()

,input_text,output_text
0,ایسی کہانی لکھیں جس میں ایک عورت اپنی برابری ا...,شام دبے پاؤں رینگ رہی تھی۔ ٹیلے پر درختوں کے س...
1,یہ کہانی وجے رانی کی ہے جو برابری کی ڈھونڈ میں...,شام دبے پاؤں رینگ رہی تھی۔ ٹیلے پر درختوں کے س...
2,ایسی کہانی لکھو جس میں ایک عورت اپنی برابری او...,شام دبے پاؤں رینگ رہی تھی۔ ٹیلے پر درختوں کے س...
3,ایسی کہانی لکھیں جس میں ایک نوجوان لڑکی، جو قح...,جموں توی کے راستے کشمیر جائیے تو کُد کے آگے ای...
4,یہ کہانی ایک بنگالی عورت سکینہ کی زندگی کے گرد...,جموں توی کے راستے کشمیر جائیے تو کُد کے آگے ای...


In [16]:
final_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Act as Intelligent Assistant and an expert to generate coherent stories in urdu, Read this Urdu_Prompt and generate a Urdu Story from this.

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(row):
    input_text = row["input_text"]
    output_text = row["output_text"]
    # Must add EOS_TOKEN to ensure proper tokenization and termination
    text = final_prompt.format(input_text, output_text) + EOS_TOKEN
    return text




In [17]:
train_df["text"] = train_df.apply(formatting_prompts_func, axis=1)

In [18]:
from datasets import Dataset

dataset = Dataset.from_pandas(train_df[['text']])


In [19]:
dataset

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 2695
})

In [20]:
print(dataset['text'][100])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Act as Intelligent Assistant and an expert to generate coherent stories in urdu, Read this Urdu_Prompt and generate a Urdu Story from this.

### Input:
یہ کہانی ایک نوجوان لڑکی جسوتی کی زندگی کے بارے میں ہے جو رواج کی ظالم اور بے پناہ انگلیوں کا شکار ہو کر اپنی زندگی کی پنکھڑی کو نوچنے پر مجبور ہو جاتی ہے۔ جسوتی کی زندگی کے واقعات اس کے آس پاس کے لوگوں کی بے حسی اور بے رحمی کی وجہ سے تباہ ہو جاتے ہیں۔ کہانی میں جسوتی کی بہن اس کی زندگی کے بارے میں سوچتی ہے اور اسے بچانے کی کوشش کرتی ہے۔ کہانی کو مکمل کریں۔

### Response:
ابھی تو زردی ہے رخ پہ کم کم،ابھی سے روتے ہیں سارے ہمدم یونہی جو چندے رہی تپ غم، تو پھر لہو بھی نہیں رہے گا اسے لاکر اس کی خواب گاہ میں لٹا دیا گیا۔ رات گرم تھی اور ویران۔ اس کی خواب گاہ کی دیواریں ہلکے آسمانی رنگ کی تھیں۔ اس پر سیاہ رنگ میں چینی کسانوں کی تصویریں پینٹ کی ہوئی تھیں جوچائے کے کھیت 

In [21]:
total_tokens=0
max_len = 0

for i in dataset['text']:
  x = str(i)
  total_tokens += len(x)
  if len(x) > max_len:
    max_len = len(x)

total_tokens , max_len


(48030013, 104895)

In [22]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 8192,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 500,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/2695 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/2695 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/2695 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/2695 [00:00<?, ? examples/s]

In [23]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

411

In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,695 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient Accumulation steps = 4
\        /    Total batch size = 4 | Total steps = 500
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.374300
20,1.332200
30,1.313000
40,1.285200
50,1.279700
60,1.241900
70,1.243000
80,1.221700
90,1.251500
100,1.205800


In [25]:
model.push_to_hub_merged("sarmadsiddiqui29/Llama-3.1-8B-Instruct-Urdu-Story", tokenizer, save_method = "merged_16bit", token = "")
tokenizer.push_to_hub("sarmadsiddiqui29/Llama-3.1-8B-Instruct-Urdu-Story", tokenizer, token = "")


Unsloth: You are pushing to hub, but you passed your HF username = sarmadsiddiqui29.
We shall truncate sarmadsiddiqui29/Llama-3.1-8B-Instruct-Urdu-Story to Llama-3.1-8B-Instruct-Urdu-Story
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 51.54 out of 83.48 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:00<00:00, 57.18it/s]


Unsloth: Saving tokenizer...

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/sarmadsiddiqui29/Llama-3.1-8B-Instruct-Urdu-Story


README.md:   0%|          | 0.00/627 [00:00<?, ?B/s]

In [26]:
from unsloth import FastLanguageModel
import torch

dtype = None
load_in_4bit = False


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "sarmadsiddiqui29/Llama-3.1-8B-Instruct-Urdu-Story",
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "",
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.50.0.dev0.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [27]:
FastLanguageModel.for_inference(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
    

In [28]:
import random

random_indices = random.sample(range(len(dataset)), 5)
filtered_dataset = train_df['input_text'][random_indices]

In [29]:
filtered_dataset

,input_text
232,یہ کہانی ایک نوجوان خاتون کی سیر و سیاحت کے اش...
1717,کہانی ایک شخص کی اپنے کتے سے گہری محبت اور اس ...
543,ایسی کہانی لکھیں جس میں ایک شخص اپنے پرانے اور...
1355,ایسی کہانی لکھو جس میں ایک مسافر بنارس پہنچتا ...
880,یہ کہانی ایک کوٹھی اور اس کے باغ کے گرد گھومتی...
